INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [3]:
import os
import time
name =  "Faiss-EM-OPUS-"+str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)

/home/lw/clc_fairseq/Faiss-EM-OPUS-20210112-1013


In [4]:
!mkdir data_bin
!cd data_bin && wget https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip && unzip  'opus-100-corpus.zip' && ls -al

--2021-01-12 10:13:25--  https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip [following]
--2021-01-12 10:13:26--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30690851 (29M) [application/zip]
Saving to: ‘opus-100-corpus.zip’

opus-100-corpus.zip 100%[===================>]  29,27M  5,54MB/s    in 5,4s    

2021-01-12 10:13:35 (5,46 MB/s) - ‘opus-100-corpus.zip’ saved [30690851/30690851]

Archive:  opus-100-corpus.zip
  inflating

In [11]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
lst_en = []
lst_vi = []

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

1004000
1004000


In [1]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[-1])
print(vi_train[-1])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

NameError: name 'lst_en' is not defined

In [14]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 75200
drwxrwxr-x 3 lw lw     4096 Thg 1  12 10:30 .
drwxrwxr-x 8 lw lw     4096 Thg 1  12 10:30 ..
drwxrwxr-x 2 lw lw     4096 Thg 1  12 10:13 data_bin
-rw-rw-r-- 1 lw lw  3318348 Thg 1  12 10:30 en_test
-rw-rw-r-- 1 lw lw 26563373 Thg 1  12 10:30 en_train
-rw-rw-r-- 1 lw lw  3328557 Thg 1  12 10:30 en_valid
-rw-rw-r-- 1 lw lw  4365721 Thg 1  12 10:30 vi_test
-rw-rw-r-- 1 lw lw 35019159 Thg 1  12 10:30 vi_train
-rw-rw-r-- 1 lw lw  4382771 Thg 1  12 10:30 vi_valid


In [15]:
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('../wiki_unigrams.bin')

In [16]:
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = open(srcfile, "r").readlines()
lst_tarfile = open(tarfile, "r").readlines()

vec_lst_srcfile = model.embed_sentences(lst_srcfile)

Test Cosin similarity

In [17]:
import faiss

vec_lst_srcfile = model.embed_sentences(lst_srcfile)
vec_lst_srcfile2 = model.embed_sentences(lst_srcfile)
x = vec_lst_srcfile
q = vec_lst_srcfile2
index = faiss.IndexFlatL2(600)
# index = faiss.index_factory(3, "Flat", faiss.METRIC_INNER_PRODUCT)
index.ntotal
faiss.normalize_L2(x)
index.add(x)
faiss.normalize_L2(q)
k = 6
distance, index = index.search(q, k)
print(index)
print(distance)

[[     0   1599  76308 133448  82410  27847]
 [     1  99160 354833 267109  59187  44527]
 [ 63365 135969 150396 270479 239117  62418]
 ...
 [803195  53990  66032  45540  76992  68747]
 [145743  43210 292962 300496 296145 180811]
 [151217 153423 112856 803197 244142 597120]]
[[7.1525574e-07 7.1525574e-07 7.1525574e-07 7.1525574e-07 7.1525574e-07
  7.1525574e-07]
 [1.1920929e-07 3.1163800e-01 3.7609673e-01 3.9664578e-01 4.7298753e-01
  5.5491567e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00]
 ...
 [1.1920929e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07
  2.3841858e-07]
 [4.7683716e-07 4.7683716e-07 4.7683716e-07 4.7683716e-07 4.7683716e-07
  4.7683716e-07]
 [2.3841858e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07 2.3841858e-07
  8.9151144e-02]]


In [18]:
i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])

a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

1422
0.0
0.0


In [19]:
import time

start = time.time()
saperate = " || "
threshold = 0.8

out_file_write = open("en_train_EM", 'w')
out_file_write_score = open("en_train_EM_score", 'w')
for i in range(len(lst_srcfile)):
    score = distance[i][1]
    if score >= threshold:
        out_file_write.writelines(lst_srcfile[i].strip() + saperate + lst_tarfile[index[i][1]])
    else:
        out_file_write.writelines(lst_srcfile[i])
    out_file_write_score.writelines(str(score) + "\n")
        

out_file_write.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

=== Ending, Total time (second):  7.957433223724365
